# Lab4. Train your Tensorflow script on SageMaker

In [5]:
from PIL import Image
import json
import os
import matplotlib.image as mpimg
import tensorflow as tf
# import pipeline
# import charset
# import model_fn
# import filters
from tensorflow.contrib.eager.python import tfe
from sagemaker.tensorflow import TensorFlow

In [6]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::308961792850:role/service-role/AmazonSageMaker-ExecutionRole-20180531T094549
CPU times: user 102 ms, sys: 12.1 ms, total: 114 ms
Wall time: 161 ms


In [7]:
bucket = None #'leonkang-datalake-seoul' # custom bucket name.
prefix = 'anpr'
assert bucket != None

In [13]:
%%time

train_channel = prefix + '/tf/tf-train'
validation_channel = prefix + '/tf/tf-validation'
train_annotation_channel = prefix + '/tf/tf-train_annotation'
validation_annotation_channel = prefix + '/tf/tf-validation_annotation'

# sess.upload_data(path='tf/tf-train', bucket=bucket, key_prefix=train_channel)
# sess.upload_data(path='tf/tf-validation', bucket=bucket, key_prefix=validation_channel)
sess.upload_data(path='tf/tf-train_annotation', bucket=bucket, key_prefix=train_annotation_channel)
sess.upload_data(path='tf/tf-validation_annotation', bucket=bucket, key_prefix=validation_annotation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(bucket, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(bucket, validation_annotation_channel)

CPU times: user 31.3 s, sys: 1.56 s, total: 32.9 s
Wall time: 2min 15s


In [14]:
!aws s3 ls leonkang-datalake-seoul/anpr/tf/tf-train/ |wc -l
!aws s3 ls leonkang-datalake-seoul/anpr/tf/tf-train_annotation/ |wc -l
!aws s3 ls leonkang-datalake-seoul/anpr/tf/tf-validation/ |wc -l
!aws s3 ls leonkang-datalake-seoul/anpr/tf/tf-validation_annotation/ |wc -l

4000
4000
1000
1001


In [15]:
s3_output_location = 's3://{}/{}/tf-output'.format(bucket, prefix)

#### Test and validation data folders
1) In this notebook
- train images: './tf-train/imgs'
- train annotations: './tf-train/annotations'
- validation images: './tf-validation/imgs'
- validation annotations: './tf-validation/annotations'

2) S3
- train images: 's3://leonkang-datalake-seoul/anpr/tf/tf-train'
- train annotations: 's3://leonkang-datalake-seoul/anpr/tf/tf-validation'
- validation images: 's3://leonkang-datalake-seoul/anpr/tf/tf-train_annotations'
- validation annotations: 's3://leonkang-datalake-seoul/anpr/tf/tf-validation_annotations'

3) Training container
- train images: '/opt/ml/input/data/train/'
- train annotations: '/opt/ml/input/data/train_annotation/'
- validation images: '/opt/ml/input/data/validation/'
- validation annotations: '/opt/ml/input/data/validation_annotation/'

In [31]:
train_data = 's3://leonkang-datalake-seoul/anpr/tf/tf-train'
validation_data = 's3://leonkang-datalake-seoul/anpr/tf/tf-validation'
train_annotation = 's3://leonkang-datalake-seoul/anpr/tf/tf-train_annotation'
validation_annotation = 's3://leonkang-datalake-seoul/anpr/tf/tf-validation_annotation'

data_channels = {'train': train_data, 'validation': validation_data, 
                 'train_annotation': train_annotation, 'validation_annotation':validation_annotation}

In [47]:
# model_dir = '/opt/ml/model'
train_instance_type='ml.p3.16xlarge'
hyperparameters = {'epochs': 30, 
                   'batch_size': 128,
                   'learning_rate': 0.001}

* Refer to API specs of SDK site -> https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/README.rst
* Refer to the API document -> https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html  
  (Note that checkpoint_path, training_steps, evaluation_steps, requirements_file args are not permitted when using Script Mode)

In [49]:
%%time
ocr_estimator = TensorFlow(entry_point='cnn_ocr.py',
                             role=role,
                             model_dir=s3_output_location,
                             hyperparameters=hyperparameters,
                             train_instance_count=2,
                             train_instance_type=train_instance_type,
                             distributions={'parameter_server': {'enabled': True}},
                             framework_version='1.12', 
                             py_version='py3')

ocr_estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2019-03-03-14-29-58-825


2019-03-03 14:29:59 Starting - Starting the training job...
2019-03-03 14:30:00 Starting - Launching requested ML instances......
2019-03-03 14:31:01 Starting - Preparing the instances for training.........
2019-03-03 14:32:34 Downloading - Downloading input data...
2019-03-03 14:33:28 Training - Downloading the training image..
2019-03-03 14:33:35,154 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-03-03 14:33:35,201 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-03-03 14:33:35,201 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2019-03-03 14:33:35,201 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-03-03 14:33:35,475 sagemaker_tensorflow_container.training INFO     Launching worker process
2019-03-03 14:33:35,833 sagemaker-containers INFO     Invoking user script

Training Env:

{

In [50]:
ocr_estimator.model_data

's3://sagemaker-ap-northeast-2-308961792850/sagemaker-tensorflow-scriptmode-2019-03-03-14-29-58-825/output/model.tar.gz'

In [51]:
!aws s3 cp {ocr_estimator.model_data} ./model/model.tar.gz --region us-east-1

download: s3://sagemaker-ap-northeast-2-308961792850/sagemaker-tensorflow-scriptmode-2019-03-03-14-29-58-825/output/model.tar.gz to model/model.tar.gz


In [74]:
!tar -xvzf ./model/model.tar.gz -C ./model

1551623990/
1551623990/variables/
1551623990/variables/variables.data-00000-of-00001
1551623990/variables/variables.index
1551623990/variables/checkpoint
1551623990/assets/
1551623990/assets/saved_model.json
1551623990/saved_model.pb
weights.ckpt.data-00000-of-00001
checkpoint
weights.ckpt.index
weights.ckpt.index
checkpoint
1551623997/
1551623997/saved_model.pb
1551623997/variables/
1551623997/variables/checkpoint
1551623997/variables/variables.data-00000-of-00001
1551623997/variables/variables.index
1551623997/assets/
1551623997/assets/saved_model.json
weights.ckpt.data-00000-of-00001


In [76]:
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = tf.keras.Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = tf.keras.layers.ZeroPadding2D((2, 2))(X_input)

    # CONV0 -> BN -> RELU -> MAXPOOL applied to X
    X = tf.keras.layers.Conv2D(48, (5, 5), strides = (1, 1), name = 'conv0')(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn0')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((2, 2), strides = (2, 2), name='max_pool0')(X)

    # CONV1 -> BN -> RELU -> MAXPOOL applied to X
    X = tf.keras.layers.Conv2D(64, (5, 5), strides = (1, 1), name = 'conv1')(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn1')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((1, 2), strides = (2, 2), name='max_pool1')(X)

    # CONV2 -> BN -> RELU -> MAXPOOL applied to X
    X = tf.keras.layers.Conv2D(64, (5, 5), strides = (1, 1), name = 'conv2')(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn2')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((2, 2), strides = (2, 2), name='max_pool2')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = tf.keras.layers.Flatten()(X)
#     X = tf.keras.layers.Dense(7, activation='softmax', name='fc')(X)

    #----------------------------------------------------------
    d1 = tf.keras.layers.Dense(81, activation='softmax', name='d1')(X)
    d2 = tf.keras.layers.Dense(81, activation='softmax', name='d2')(X)
    d3 = tf.keras.layers.Dense(81, activation='softmax', name='d3')(X)
    d4 = tf.keras.layers.Dense(81, activation='softmax', name='d4')(X)
    d5 = tf.keras.layers.Dense(81, activation='softmax', name='d5')(X)
    d6 = tf.keras.layers.Dense(81, activation='softmax', name='d6')(X)
    d7 = tf.keras.layers.Dense(81, activation='softmax', name='d7')(X)
    
    model = tf.keras.Model(inputs = X_input, 
                  outputs = [d1,d2,d3,d4,d5,d6,d7],
                  name='CNN_ANPR')

    return model

In [80]:
tf.keras.backend.clear_session()
device = '/cpu:0' 

with tf.device(device):    
    model = model((128,64,1))
    saver = tfe.Saver(model.variables)
    saver.restore('model/weights.ckpt')

ValueError: Shape must be rank 4 but is rank 1 for 'CNN_ANPR/zero_padding2d/Pad' (op: 'Pad') with input shapes: [3], [4,2].

In [52]:
predictor = ocr_estimator.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge',
                                 accelerator_type='ml.eia1.medium')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-scriptmode-2019-03-03-14-29-58-825
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-scriptmode-2019-03-03-14-29-58-825


----------------------------------------------------------------!

In [53]:
test_imgs = os.listdir('tf/tf-validation')
threshold = 0.4

In [71]:
%%time
for i in test_imgs[:10]:
    ifile = 'tf/tf-validation/' + i
    with open(ifile, "rb") as image:
        f = image.read()
        b = bytearray(f)
        results = predictor.predict(b)
#         detections = json.loads(results)
#         print(detections)
#     visualize_detection(ifile, detections['prediction'], 'plate', threshold)

TypeError: Object of type 'bytearray' is not JSON serializable

In [ ]:
%%time
for i in test_imgs[:10]:
    ifile = 'od/od-validation/' + i
    with open(ifile, "rb") as image:

In [66]:
test_img = Image.open('tf/tf-validation/'+test_imgs[0])

In [69]:
type(test_img)

PIL.PngImagePlugin.PngImageFile

In [65]:
test_imgs[0]
#predictor.predict()

'00004479_89_8910_0.png'

In [63]:
import boto3
runtime = boto3.Session().client('runtime.sagemaker')

In [73]:
tf_endpoint = 'sagemaker-tensorflow-scriptmode-2019-03-03-14-29-58-825'
response = runtime.invoke_endpoint(EndpointName=tf_endpoint,
                                   ContentType='image/png',
                                   Body=b)
response['Body']

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: <PIL.PngImagePlugin.PngImageFile image mode=L size=72x23 at 0x7F688A924390>, type: <class 'PIL.PngImagePlugin.PngImageFile'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object

In [ ]:
temp = response['Body'].read()

In [ ]:
import numpy as np
def get_imgs_data(train_dir):
    ck_imgs = os.listdir(train_dir)
    flg_first = True
    for i in ck_imgs:
        ext = i.split('.')[1]
        if ext == 'png':
#             print(train_dir + i)
            img=Image.open(train_dir + i) 
            img = img.resize((128, 64))
            imgs = img if flg_first else np.append(imgs, img)
            flg_first = False
    return imgs.reshape([-1, 128, 64, 1])/255.
imgs = get_imgs_data('tf-validation/')

In [ ]:
results = predictor.predict(imgs[:1])['predictions'] 
# flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
# print('predictions: \t{}'.format(np.array(flat_list)))
# print('target values: \t{}'.format(y_test[:10].round(decimals=1)))
Y = results[0]
print(np.argmax(Y['d1']))